In [ ]:
!git clone https://github.com/clemsadand/AirPollution.git

In [ ]:
%cd AirPollution

In [ ]:
# !apt-get update -q
# !apt-get install -y libglu1-mesa -q
# !pip install gmsh numpy matplotlib meshio -q
# !pip install pyDOE -q

In [ ]:
!sudo apt-get update -q
!sudo apt-get install -y libglu1-mesa -q
!pip install -r requirements.txt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# n_dofs = [45, 191, 846, 3431, 13998, 56408, 226206]
# for i, n_ in enumerate(n_dofs):
#     if i<=5:
#         layers = [3] + [32] * (1+i) + [1]
#     else:
#         layers = [3] + [60] * (i) + [1]
#     n_params = sum(l1 * l2 + l2 for l1, l2 in zip(layers[:-1], layers[1:]))
#     print(f"{n_} --:-- {n_params}")
#     # print((n_dofs[i] - n_params)/n_dofs[i])
#     # print((n_-1)/5)
#     print()

In [ ]:
import numpy as np

def twolayers(w):
    return w**2 + 6*w+1

def threelayers(w):
    return 2*w**2 + 7*w+1

def fourlayers(w):
    return 3*w**2 + 8*w + 1

def cost(w, n_dofs, ratio, loss_fn):
    return np.abs(loss_fn(w) / n_dofs - ratio)

n_dofs_vals = [45, 191, 846, 3431, 13998, 56408]
ratio = 0.09

w_vals = np.arange(2, 1000)
for n_dofs in n_dofs_vals:
    loss_fn = fourlayers
    cost_vals = cost(w_vals, n_dofs, ratio, loss_fn)

    max_cost = np.min(cost_vals)
    smallest_w_for_max = w_vals[np.argmin(cost_vals)]

    print(f"Max cost: {max_cost}")
    print(f"Smallest w for max cost: {smallest_w_for_max}")
    print(f"Number of params: {loss_fn(smallest_w_for_max)}")
    print()

In [ ]:
fourlayers(2)

In [ ]:
[45, 191, 846, 3431, 13998, 56408]

In [ ]:
n_neurons = [2, 4, 8, 16, 32, 64]

In [ ]:
# --- Imports ---
import numpy as np
import crbe_fem
import pinn2
import meshio
from tqdm import tqdm
import time
import torch
import psutil
import pandas as pd

# --- Function to track GPU and CPU memory ---
def get_gpu_memory():
    if torch.cuda.is_available():
        return torch.cuda.max_memory_allocated() / 1e6  # in MB
    return 0

def get_cpu_memory():
    return psutil.Process().memory_info().rss / 1e6  # in MB

# --- Problem Setup ---
domain_size = 20.0
Lx = Ly = domain_size
T = 10.0
D = 0.1
vx, vy = 1.0, 0.5
sigma = 0.1

domain_params = crbe_fem.DomainParams(Lx=Lx, Ly=Ly, T=T)
model_params = crbe_fem.Models(vx=vx, vy=vy, D=D, sigma=sigma)

t_domain = [0.0, T]
x_domain = [-domain_size, domain_size]
y_domain = [-domain_size, domain_size]
lb = np.array([t_domain[0], x_domain[0], y_domain[0]])
ub = np.array([t_domain[1], x_domain[1], y_domain[1]])

domain = [-20, 20, -20, 20]
time_range = [0, 10]
v = [vx, vy]

lambda_weights = {'pde': 1.0, 'ic': 10.0, 'bc': 10.0}
learning_rate = 0.001
epochs = 20000
n_steps = 128
T_final = T

# --- Experimental Settings ---
mesh_sizes = [4, 8, 16, 32, 64, 128]
n_neurons = [2, 4, 8, 16, 32, 64]

# --- Logging ---
n_dofs = []
n_boundary_dofs = []
pinn_results = []
result_history = {}

# --- Main Loop ---
for i, mesh_size in enumerate(mesh_sizes):
    # Reset GPU memory tracking
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.empty_cache()
    initial_cpu_memory = get_cpu_memory()
    initial_gpu_memory = get_gpu_memory()

    layers = [3] + [n_neurons[i]] * 4 + [1]
    mesh_file = crbe_fem.create_mesh(mesh_size, domain_size=domain_size)
    mesh = meshio.read(mesh_file)
    mesh_data = crbe_fem.MeshData(mesh, domain_params, nt=n_steps)

    n_dofs.append(mesh_data.number_of_segments)
    n_boundary_dofs.append(len(mesh_data.boundary_segments))

    n_ic = round(0.2 * mesh_data.number_of_segments)
    n_bc = n_ic
    n_col = mesh_data.number_of_segments - n_ic - n_bc
    batch_sizes = {'pde': n_col, 'ic': n_ic, 'bc': n_ic}

    model = pinn2.PINN(layers, D, v)

    print(f"Training for mesh size {mesh_size} ...")
    start_time = time.time()
    history = pinn2.train_pinn(model, domain, time_range, batch_sizes, learning_rate, epochs, lambda_weights)
    train_time = time.time() - start_time

    final_gpu_memory = get_gpu_memory()
    final_cpu_memory = get_cpu_memory()

    result_history[f"mesh_size_{mesh_size}"] = history

    x_eval_flat = mesh_data.midpoints[:,0].reshape(-1, 1)
    y_eval_flat = mesh_data.midpoints[:,1].reshape(-1, 1)
    t_eval = np.ones_like(x_eval_flat) * T_final

    rel_l2_error, max_error, u_pred, u_exact = pinn2.compute_error(
        model, t_eval, x_eval_flat, y_eval_flat
    )

    pinn_results.append({
        "time": T_final,
        "rel_l2_error": rel_l2_error,
        "max_error": max_error,
        "mesh_size": mesh_size,
        "train_time": train_time,
        "final_loss": history["total_loss"][-1],
        "number_of_collocation_points": mesh_data.number_of_segments,
        "n_parameters": sum(l1 * l2 + l2 for l1, l2 in zip(layers[:-1], layers[1:])),
        "gpu_memory_usage_MB": final_gpu_memory - initial_gpu_memory,
        "cpu_memory_usage_MB": final_cpu_memory - initial_cpu_memory,
    })

    print(f"Mesh size: {mesh_size}")
    print(f"GPU Memory: {final_gpu_memory - initial_gpu_memory:.2f} MB")
    print(f"CPU Memory: {final_cpu_memory - initial_cpu_memory:.2f} MB")
    print("-" * 40)

    del model

# --- Export Results ---
df_pinn = pd.DataFrame(pinn_results)
df_pinn

# # Optional: Visualize
# import matplotlib.pyplot as plt
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(df_pinn['mesh_size'], df_pinn['gpu_memory_usage_MB'], 'o-', label='GPU Memory')
# plt.xlabel('Mesh Size'); plt.ylabel('Memory Usage (MB)')
# plt.title('GPU Memory Usage'); plt.grid(True); plt.xscale('log')

# plt.subplot(1, 2, 2)
# plt.plot(df_pinn['mesh_size'], df_pinn['cpu_memory_usage_MB'], 'o-', label='CPU Memory')
# plt.xlabel('Mesh Size'); plt.ylabel('Memory Usage (MB)')
# plt.title('CPU Memory Usage'); plt.grid(True); plt.xscale('log')

# plt.tight_layout()
# plt.savefig('memory_usage.png')
# plt.show()


In [ ]:
# import pandas as pd

# df_pinn = pd.DataFrame(pinn_results)
df_pinn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import crbe_fem
import pinns
import meshio
from tqdm import tqdm
import time
import psutil
import torch
import pandas as pd
import gc

# --- Problem Setup ---
domain_size = 20.0
Lx = Ly = domain_size
T = 10.0
D = 0.1
vx, vy = 1.0, 0.5
sigma = 0.1

domain_params = crbe_fem.DomainParams(Lx=Lx, Ly=Ly, T=T)
model_params = crbe_fem.Models(vx=vx, vy=vy, D=D, sigma=sigma)

t_domain = [0.0, T]
x_domain = [-domain_size, domain_size]
y_domain = [-domain_size, domain_size]

lb = np.array([t_domain[0], x_domain[0], y_domain[0]])
ub = np.array([t_domain[1], x_domain[1], y_domain[1]])

exact_solutions = {
    "name": "Impulsion gaussienne",
    "func": lambda t, x, y: pinns.exact_solution_gaussian_pulse(t, x, y, D, [vx, vy]),
    "epochs": 1000,
    "lr": 3e-4
}

exact_sol_name = exact_solutions["name"]
exact_sol_fn = exact_solutions["func"]
epochs = exact_solutions["epochs"]
lr = exact_solutions["lr"]

mesh_sizes = [4, 8, 16, 32, 64, 128]
n_steps = 128
crbe_results = []

# --- Function to track CPU memory ---
def get_cpu_memory():
    return psutil.Process().memory_info().rss / 1e6  # in MB

# --- Loop over mesh sizes ---
for i, mesh_size in enumerate(mesh_sizes):
    # --- Prepare for memory tracking ---
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

    initial_cpu_memory = get_cpu_memory()

    print(f"Training for mesh size = {mesh_size} ...")
    start_time = time.time()

    # --- Mesh and Solver Setup ---
    mesh_file = crbe_fem.create_mesh(mesh_size, domain_size=domain_size)
    mesh = meshio.read(mesh_file)
    mesh_data = crbe_fem.MeshData(mesh, domain_params, nt=n_steps)

    solver = crbe_fem.BESCRFEM(domain_params, model_params, mesh_data, use_quadrature=True)

    solver.solve()
    errors = solver.compute_errors()
    train_time = time.time() - start_time

    # --- Memory tracking after solve ---
    gc.collect()
    final_cpu_memory = get_cpu_memory()
    final_gpu_memory = torch.cuda.max_memory_allocated() / 1e6 if torch.cuda.is_available() else 0

    # --- Save results ---
    crbe_results.append({
        "solution": exact_sol_name,
        "time": T,
        "rel_l2_error": errors['final_l2_error'],
        "max_error": errors['final_linf_error'],
        "mesh_size": mesh_size,
        "train_time": train_time,
        "gpu_memory_usage_MB": final_gpu_memory,
        "cpu_memory_usage_MB": final_cpu_memory - initial_cpu_memory,
        "number_of_collocation_points": mesh_data.number_of_segments,
    })

    # --- Print summary ---
    print(f"Mesh size: {mesh_size}")
    print(f"Peak GPU Memory: {final_gpu_memory:.2f} MB")
    print(f"CPU Memory Used: {final_cpu_memory - initial_cpu_memory:.2f} MB")
    print("-" * 40)

# --- Results as DataFrame ---
df_crbe = pd.DataFrame(crbe_results)
display(df_crbe)

# --- Plot Memory Usage ---
plt.figure(figsize=(10, 6))
plt.plot(mesh_sizes, df_crbe['gpu_memory_usage_MB'], label='GPU Memory (MB)', marker='o')
plt.plot(mesh_sizes, df_crbe['cpu_memory_usage_MB'], label='CPU Memory (MB)', marker='s')
plt.xlabel('Mesh Size')
plt.ylabel('Memory Usage (MB)')
plt.title('Memory Usage vs Mesh Size')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df_pinn.to_csv("df_pinn_training_results.csv")
df_crbe.to_csv("df_crbe_training_results.csv")

# import pandas as pd
# df_pinn = pd.read_csv("df_pinn_training_results.csv")
# df_crbe = pd.read_csv("df_crbe_training_results.csv")

## Sensitivity Analysis

In [ ]:
import numpy as np
import crbe_fem
import pinn2
import meshio
from tqdm import tqdm
import time



# Problem setup
domain_size = 20
domain = [-domain_size, domain_size, -domain_size, domain_size]  # [x_min, x_max, y_min, y_max] in km
T_final = 10
time_range = [0, T_final]  # [t_min, t_max] in seconds
v = [1.0, 0.5]  # Velocity vector in m/s
D_list = [0.01, 0.1, 1.0]  # List of Diffusion coefficient in m²/s
v = [1.0, 0.5]
sigma = 0.1

# Résultats
pinn_results = []
pinn_histories = {}
crbe_results = []

# ======== FIxed parameters
mesh_size = 256
n_steps = 128

#
domain_params = crbe_fem.DomainParams(Lx=domain_size, Ly=domain_size, T=T_final)



#mesh
mesh_file = crbe_fem.create_mesh(mesh_size, domain_size=domain_size)
mesh = meshio.read(mesh_file)
mesh_data = crbe_fem.MeshData(mesh, domain_params, nt=n_steps)

n_ic = mesh_size
n_bc = mesh_size
n_col = mesh_data.number_of_segments - n_ic - n_bc


lambda_weights = {'pde': 1.0, 'ic': 10.0, 'bc': 10.0}
learning_rate = 0.001
batch_sizes = {'pde': n_col, 'ic': n_ic, 'bc': n_bc}
epochs = 10000
layers = [3, 20, 20, 20, 20, 20, 1]

sensitivity_data = []

for i, nu in enumerate(D_list):
    #============================================================
    #PINN's setup
    model = pinn2.PINN(layers, nu, v)

    print(f"Training for {mesh_size} data ...")
    
    start_time = time.time()
    history = pinn2.train_pinn(model, domain, time_range, batch_sizes, learning_rate, epochs, lambda_weights)
    train_time = time.time() - start_time

    pinn_histories[f"mesh_size_{mesh_size}"] = history

    x_eval_flat = mesh_data.midpoints[:,0].reshape(-1, 1)
    y_eval_flat = mesh_data.midpoints[:,1].reshape(-1, 1)
    t_eval = np.ones_like(x_eval_flat) * T_final

    rel_l2_error, max_error, u_pred, u_exact = pinn2.compute_error(
                model, t_eval, x_eval_flat, y_eval_flat)

    # # Ajout des résultats
    # sensitivity_data.append({
    #     "method": "PINN",
    #     "diffusion": nu,
    #     "time": T_final,
    #     "rel_l2_error": rel_l2_error,
    #     "max_error": max_error,
    #     "mesh_size": mesh_size,
    #     "train_time": train_time,
    #     "final_loss": history["total_loss"][-1],
    #     "number_of_collocation_points": mesh_data.number_of_segments,
    #     "n_parameters": sum(l1 * l2 + l2 for l1, l2 in zip(layers[:-1], layers[1:])),
    # })
    
    #=============================================================
    #CRBE's setup
    exact_sol_fn = lambda t, x, y: model.f_analytical(t, x, y)

    #model
    model_params = crbe_fem.Models(vx=v[0], vy=v[1], D=nu, sigma=sigma)

    print(f"Training for {mesh_size} data ...")
    start_time = time.time()
    solver = crbe_fem.BESCRFEM(domain_params, model_params, mesh_data, use_quadrature=True)
    solver.solve()

    train_time = time.time() - start_time
    errors = solver.compute_errors()

     # Ajout des résultats
    sensitivity_data.append({
        "diffusion_coef": nu,
        "pinn_l2_error": rel_l2_error,
        "max_error": max_error,
        "cr_l2_error": errors['final_l2_error'],
        "cr_max_error": errors['final_linf_error'],
    })


In [ ]:
df_sensitivity_data = pd.DataFrame(sensitivity_data)
# df_sensitivity_data = pd.DataFrame(sensitivity_data)

In [ ]:
df_sensitivity_data

## Comprehensive Dashboard

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Create a figure with two subplots for L2 and Linf errors
# plt.figure(figsize=(14, 10))

# Plot 1: L2 Error
plt.figure(figsize=(10, 6))

plt.loglog(df_crbe['mesh_size'], df_crbe['rel_l2_error'], 'o-', linewidth=2, markersize=8, label='CR-BE (L2)')
plt.loglog(df_pinn['mesh_size'], df_pinn['rel_l2_error'], 's-', linewidth=2, markersize=8, label='PINN (L2)')

# Add reference lines for convergence rates
x = np.array([4, 128])
plt.loglog(x, 1/x, 'k--', alpha=0.7, label='O(h¹)')
plt.loglog(x, 1/x**2, 'k-.', alpha=0.7, label='O(h²)')

plt.xlabel('Mesh Size', fontsize=12)
plt.ylabel('Relative L2 Error', fontsize=12)
plt.title('Relative L2 Error Comparison: CR-BE vs PINN', fontsize=14)
plt.grid(True, which="both", ls="--", alpha=0.7)
plt.legend(fontsize=10)

plt.tight_layout()
plt.savefig('error_comparison_loglog_l2.png', dpi=300, bbox_inches='tight')
plt.savefig('error_comparison_loglog_l2.pdf', dpi=300, bbox_inches='tight')
plt.show()

# Plot 2: Linf Error
plt.figure(figsize=(10, 6))
plt.loglog(df_crbe['mesh_size'], df_crbe['max_error'], 'o-', linewidth=2, markersize=8, label='CR-BE (L∞)')
plt.loglog(df_pinn['mesh_size'], df_pinn['max_error'], 's-', linewidth=2, markersize=8, label='PINN (L∞)')

# Add reference lines for convergence rates
plt.loglog(x, 1/x, 'k--', alpha=0.7, label='O(h¹)')
plt.loglog(x, 1/x**2, 'k-.', alpha=0.7, label='O(h²)')

plt.xlabel('Mesh Size', fontsize=12)
plt.ylabel('Sup Error (L∞)', fontsize=12)
plt.title('Sup Error (L∞) Comparison: CR-BE vs PINN', fontsize=14)
plt.grid(True, which="both", ls="--", alpha=0.7)
plt.legend(fontsize=10)

plt.tight_layout()
plt.savefig('error_comparison_loglog_linf.png', dpi=300, bbox_inches='tight')
plt.savefig('error_comparison_loglog_linf.pdf', dpi=300, bbox_inches='tight')
plt.show()

# Additional plot: Computational efficiency (train time vs mesh size)
plt.figure(figsize=(10, 6))
plt.loglog(df_crbe['mesh_size'], df_crbe['train_time'], 'o-', linewidth=2, markersize=8, label='CR-BE')
plt.loglog(df_pinn['mesh_size'], df_pinn['train_time'], 's-', linewidth=2, markersize=8, label='PINN')

# Add reference lines for computational complexity
plt.loglog(x, x**1, 'k--', alpha=0.7, label='O(h¹)')
plt.loglog(x, x**2, 'k-.', alpha=0.7, label='O(h²)')

plt.xlabel('Mesh Size', fontsize=12)
plt.ylabel('Training Time (s)', fontsize=12)
plt.title('Computational Efficiency: CR-BE vs PINN', fontsize=14)
plt.grid(True, which="both", ls="--", alpha=0.7) 
plt.legend(fontsize=10)

plt.tight_layout()
plt.savefig('computational_efficiency_loglog.png', dpi=300, bbox_inches='tight')
plt.savefig('computational_efficiency_loglog.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Advanced Convergence Analysis

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import linregress
import seaborn as sns
from matplotlib.ticker import ScalarFormatter

# 1. Calculate actual convergence rates
def calculate_convergence_rates(df, error_column):
    mesh_sizes = df['mesh_size'].values
    errors = df[error_column].values
    
    # Calculate log values
    log_h = np.log(1/mesh_sizes)
    log_err = np.log(errors)
    
    # Linear regression to find slope (convergence rate)
    slope, intercept, r_value, p_value, std_err = linregress(log_h, log_err)
    
    return slope, r_value**2

# Calculate convergence rates for both methods
cr_l2_rate, cr_l2_r2 = calculate_convergence_rates(df_crbe, 'rel_l2_error')
pinn_l2_rate, pinn_l2_r2 = calculate_convergence_rates(df_pinn, 'rel_l2_error')
cr_linf_rate, cr_linf_r2 = calculate_convergence_rates(df_crbe, 'max_error')
pinn_linf_rate, pinn_linf_r2 = calculate_convergence_rates(df_pinn, 'max_error')

print(f"CR-BE L2 convergence rate: {cr_l2_rate:.3f} (R² = {cr_l2_r2:.3f})")
print(f"PINN L2 convergence rate: {pinn_l2_rate:.3f} (R² = {pinn_l2_r2:.3f})")
print(f"CR-BE L∞ convergence rate: {cr_linf_rate:.3f} (R² = {cr_linf_r2:.3f})")
print(f"PINN L∞ convergence rate: {pinn_linf_rate:.3f} (R² = {pinn_linf_r2:.3f})")

# 2. Error vs computational cost (efficiency analysis)
plt.figure(figsize=(10, 6))
plt.loglog(df_crbe['train_time'], df_crbe['rel_l2_error'], 'o-', linewidth=2, markersize=8, label=f'CR-BE (Rate: {cr_l2_rate:.2f})')
plt.loglog(df_pinn['train_time'], df_pinn['rel_l2_error'], 's-', linewidth=2, markersize=8, label=f'PINN (Rate: {pinn_l2_rate:.2f})')

plt.xlabel('Computational Time (s)', fontsize=12)
plt.ylabel('Relative L2 Error', fontsize=12)
plt.title('Error vs Computational Cost', fontsize=14)
plt.grid(True, which="both", ls="--", alpha=0.7)
plt.legend(fontsize=10)
plt.tight_layout()
plt.savefig('error_vs_time.png', dpi=300, bbox_inches='tight')
plt.savefig('error_vs_time.pdf', dpi=300, bbox_inches='tight')
plt.show()

# 3. Compute and plot efficiency metrics
df_crbe['efficiency'] = df_crbe['rel_l2_error'] * df_crbe['train_time']
df_pinn['efficiency'] = df_pinn['rel_l2_error'] * df_pinn['train_time']

plt.figure(figsize=(10, 6))
plt.loglog(df_crbe['mesh_size'], df_crbe['efficiency'], 'o-', linewidth=2, markersize=8, label='CR-BE')
plt.loglog(df_pinn['mesh_size'], df_pinn['efficiency'], 's-', linewidth=2, markersize=8, label='PINN')

plt.xlabel('Mesh Size', fontsize=12)
plt.ylabel('Efficiency Metric (Error × Running Time)', fontsize=12)
plt.title('Computational Efficiency Comparison', fontsize=14)
plt.grid(True, which="both", ls="--", alpha=0.7)
plt.legend(fontsize=10)
plt.tight_layout()
plt.savefig('efficiency_comparison.png', dpi=300, bbox_inches='tight')
plt.savefig('efficiency_comparison.pdf', dpi=300, bbox_inches='tight')
plt.show()

# 4. Memory usage estimation (approximate based on number of parameters/DOFs)
# Note: This is an approximation; actual memory would need profiling
if 'n_parameters' in df_pinn:
    df_crbe['memory_estimate'] = np.array(n_dofs) * 8 / 1024  # KB (8 bytes per double)
    df_pinn['memory_estimate'] = df_pinn['n_parameters'] * 4 / 1024  # KB (4 bytes per float32 parameter)
    
    plt.figure(figsize=(10, 6))
    plt.loglog(df_crbe['memory_estimate'], df_crbe['rel_l2_error'], 'o-', linewidth=2, markersize=8, label='CR-BE')
    plt.loglog(df_pinn['memory_estimate'], df_pinn['rel_l2_error'], 's-', linewidth=2, markersize=8, label='PINN')
    
    plt.xlabel('Estimated Memory Usage (KB)', fontsize=12)
    plt.ylabel('Relative L2 Error', fontsize=12)
    plt.title('Error vs Memory Usage', fontsize=14)
    plt.grid(True, which="both", ls="--", alpha=0.7)
    plt.legend(fontsize=10)
    plt.tight_layout()
    plt.savefig('error_vs_memory.png', dpi=300, bbox_inches='tight')
    plt.savefig('error_vs_memory.pdf', dpi=300, bbox_inches='tight')
    plt.show()

# 5. Visualization of solution comparison at highest resolution
# Assuming we can extract solutions at final time point on a grid

def plot_comparison_at_final_time(mesh_size_idx=-1):
    """Plot solution comparison at the final time using the specified mesh size index"""
    # This would need to be adapted based on how you store solutions
    # Here's a placeholder structure:
    
    # Extract or recompute solutions on a common grid
    n_viz = 100
    x_viz = np.linspace(-domain_size, domain_size, n_viz)
    y_viz = np.linspace(-domain_size, domain_size, n_viz)
    X_viz, Y_viz = np.meshgrid(x_viz, y_viz)
    
    # Get the mesh size for the plot
    mesh_size = mesh_sizes[mesh_size_idx]
    
    # Placeholder: You'll need to implement the solution extraction or recomputation
    # For example:
    # cr_solution = extract_solution(solver, T, X_viz, Y_viz)
    # pinn_solution = model.predict(np.column_stack((np.full(X_viz.size, T), X_viz.flatten(), Y_viz.flatten()))).reshape(X_viz.shape)
    # exact_solution = np.array([exact_sol_fn(T, x, y) for x, y in zip(X_viz.flatten(), Y_viz.flatten())]).reshape(X_viz.shape)
    
    # For demonstration, let's create some placeholder data
    # Replace these with actual solution evaluations
    t_eval = T
    exact_solution = np.zeros((n_viz, n_viz))
    cr_solution = np.zeros((n_viz, n_viz))
    pinn_solution = np.zeros((n_viz, n_viz))
    
    for i in range(n_viz):
        for j in range(n_viz):
            x, y = X_viz[i, j], Y_viz[i, j]
            exact_solution[i, j] = exact_sol_fn(t_eval, x, y)
            # You would need to implement these:
            # cr_solution[i, j] = cr_solution_at_point(t_eval, x, y)
            # pinn_solution[i, j] = pinn_solution_at_point(t_eval, x, y)
    
    # Create grid of error plots
    fig, axes = plt.subplots(2, 3, figsize=(18, 10), sharex=True, sharey=True)
    
    # Plot solutions
    levels = np.linspace(np.min(exact_solution), np.max(exact_solution), 50)
    im1 = axes[0, 0].contourf(X_viz, Y_viz, exact_solution, levels=levels, cmap='viridis')
    im2 = axes[0, 1].contourf(X_viz, Y_viz, cr_solution, levels=levels, cmap='viridis')
    im3 = axes[0, 2].contourf(X_viz, Y_viz, pinn_solution, levels=levels, cmap='viridis')
    
    axes[0, 0].set_title(f'Exact Solution at t={T}', fontsize=12)
    axes[0, 1].set_title(f'CR-BE Solution (mesh={mesh_size})', fontsize=12)
    axes[0, 2].set_title(f'PINN Solution (n={mesh_size}²)', fontsize=12)
    
    # Plot errors
    cr_error = np.abs(cr_solution - exact_solution)
    pinn_error = np.abs(pinn_solution - exact_solution)
    error_levels = np.linspace(0, max(np.max(cr_error), np.max(pinn_error)), 50)
    
    im4 = axes[1, 0].contourf(X_viz, Y_viz, np.zeros_like(X_viz), levels=error_levels, cmap='magma')  # placeholder
    im5 = axes[1, 1].contourf(X_viz, Y_viz, cr_error, levels=error_levels, cmap='magma')
    im6 = axes[1, 2].contourf(X_viz, Y_viz, pinn_error, levels=error_levels, cmap='magma')
    
    axes[1, 0].set_title('Reference (Zero Error)', fontsize=12)
    axes[1, 1].set_title(f'CR-BE Error (L∞={np.max(cr_error):.2e})', fontsize=12)
    axes[1, 2].set_title(f'PINN Error (L∞={np.max(pinn_error):.2e})', fontsize=12)
    
    # Add colorbars
    cbar1 = fig.colorbar(im1, ax=axes[0, :], orientation='horizontal', pad=0.1, aspect=30)
    cbar1.set_label('Solution Value')
    
    cbar2 = fig.colorbar(im5, ax=axes[1, :], orientation='horizontal', pad=0.1, aspect=30)
    cbar2.set_label('Absolute Error')
    
    # Add labels
    for i in range(2):
        for j in range(3):
            axes[i, j].set_xlabel('x', fontsize=10)
            axes[i, j].set_ylabel('y', fontsize=10)
    
    plt.tight_layout()
    plt.savefig('solution_comparison.png', dpi=300, bbox_inches='tight')
    plt.savefig('solution_comparison.pdf', dpi=300, bbox_inches='tight')
    plt.show()

# Uncomment this when you have the solutions available
# plot_comparison_at_final_time()

# 6. Parameter sensitivity analysis
# This would run multiple convergence tests with different parameters
# Here's a simplified example for diffusion coefficient sensitivity

def run_parameter_sensitivity_analysis():
    """Example of how to run parameter sensitivity analysis"""
    # Parameters to test
    diffusion_coefficients = [0.01, 0.1, 1.0]
    mesh_selection = [8, 32, 128]  # Selected mesh sizes to test
    
    # Results storage
    sensitivity_results = []
    
    for D_value in diffusion_coefficients:
        print(f"Testing diffusion coefficient D = {D_value}")
        
        # Update model parameters
        model_params = crbe_fem.Models(vx=vx, vy=vy, D=D_value, sigma=sigma)
        
        for mesh_size in mesh_selection:
            # Run CR-BE solver with this configuration
            mesh_file = crbe_fem.create_mesh(mesh_size, domain_size=domain_size)
            mesh = meshio.read(mesh_file)
            mesh_data = crbe_fem.MeshData(mesh, domain_params, nt=n_steps)
            
            solver = crbe_fem.BESCRFEM(domain_params, model_params, mesh_data, use_quadrature=True)
            solver.solve()
            
            errors = solver.compute_errors()
            
            # Store results
            sensitivity_results.append({
                "method": "CR-BE",
                "diffusion": D_value,
                "mesh_size": mesh_size,
                "rel_l2_error": errors['final_l2_error'],
                "max_error": errors['final_linf_error']
            })
            
            # Similarly for PINN (would need to implement)
            # ...
    
    # Convert to DataFrame and analyze
    df_sensitivity = pd.DataFrame(sensitivity_results)
    
    # Plot sensitivity results
    plt.figure(figsize=(12, 8))
    sns.lineplot(data=df_sensitivity, x='mesh_size', y='rel_l2_error', 
                 hue='diffusion', style='method', markers=True, dashes=False)
    
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Mesh Size', fontsize=12)
    plt.ylabel('Relative L2 Error', fontsize=12)
    plt.title('Sensitivity to Diffusion Coefficient', fontsize=14)
    plt.grid(True, which="both", ls="--", alpha=0.7)
    plt.legend(title='Diffusion Coef. (D)', fontsize=10)
    plt.tight_layout()
    plt.savefig('parameter_sensitivity.png', dpi=300, bbox_inches='tight')
    plt.savefig('parameter_sensitivity.pdf', dpi=300, bbox_inches='tight')
    plt.show()

# Uncomment to run sensitivity analysis (this will take time!)
# run_parameter_sensitivity_analysis()

# 7. Visualization of all result metrics in a comprehensive dashboard
def create_results_dashboard():
    """Create a comprehensive dashboard with all metrics"""
    # Combine all results
    df_combined = pd.concat([
        df_crbe.assign(method='CR-BE'),
        df_pinn.assign(method='PINN')
    ])
    
    # Create figure with multiple subplots
    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # 1. L2 Error convergence
    ax1 = fig.add_subplot(gs[0, 0])
    for method, color, marker in zip(['CR-BE', 'PINN'], ['blue', 'red'], ['o', 's']):
        df_method = df_combined[df_combined['method'] == method]
        ax1.loglog(df_method['mesh_size'], df_method['rel_l2_error'], 
                  marker=marker, linestyle='-', color=color, label=method)
    
    # Add reference lines
    x_ref = np.array([min(mesh_sizes), max(mesh_sizes)])
    ax1.loglog(x_ref, 1/x_ref, 'k--', alpha=0.5, label='O(h)')
    ax1.loglog(x_ref, 1/x_ref**2, 'k-.', alpha=0.5, label='O(h²)')
    
    ax1.set_xlabel('Mesh Size')
    ax1.set_ylabel('L2 Error')
    ax1.set_title('Error Convergence (L2)')
    ax1.grid(True, which='both', ls='--', alpha=0.6)
    ax1.legend()
    
    # 2. L∞ Error convergence
    ax2 = fig.add_subplot(gs[0, 1])
    for method, color, marker in zip(['CR-BE', 'PINN'], ['blue', 'red'], ['o', 's']):
        df_method = df_combined[df_combined['method'] == method]
        ax2.loglog(df_method['mesh_size'], df_method['max_error'], 
                  marker=marker, linestyle='-', color=color, label=method)
    
    # Add reference lines
    ax2.loglog(x_ref, 1/x_ref, 'k--', alpha=0.5, label='O(h)')
    ax2.loglog(x_ref, 1/x_ref**2, 'k-.', alpha=0.5, label='O(h²)')
    
    ax2.set_xlabel('Mesh Size')
    ax2.set_ylabel('L∞ Error')
    ax2.set_title('Error Convergence (L∞)')
    ax2.grid(True, which='both', ls='--', alpha=0.6)
    ax2.legend()
    
    # 3. Computational time
    ax3 = fig.add_subplot(gs[0, 2])
    for method, color, marker in zip(['CR-BE', 'PINN'], ['blue', 'red'], ['o', 's']):
        df_method = df_combined[df_combined['method'] == method]
        ax3.loglog(df_method['mesh_size'], df_method['train_time'], 
                  marker=marker, linestyle='-', color=color, label=method)
    
    ax3.set_xlabel('Mesh Size')
    ax3.set_ylabel('Running Time (s)')
    ax3.set_title('Computational Cost')
    ax3.grid(True, which='both', ls='--', alpha=0.6)
    ax3.legend()
    
    # 4. Error vs Time
    ax4 = fig.add_subplot(gs[1, 0])
    for method, color, marker in zip(['CR-BE', 'PINN'], ['blue', 'red'], ['o', 's']):
        df_method = df_combined[df_combined['method'] == method]
        ax4.loglog(df_method['train_time'], df_method['rel_l2_error'], 
                  marker=marker, linestyle='-', color=color, label=method)
    
    ax4.set_xlabel('Running Time (s)')
    ax4.set_ylabel('L2 Error')
    ax4.set_title('Efficiency (Error vs Time)')
    ax4.grid(True, which='both', ls='--', alpha=0.6)
    ax4.legend()
    
    # 5. Efficiency metric
    ax5 = fig.add_subplot(gs[1, 1])
    for method, color, marker in zip(['CR-BE', 'PINN'], ['blue', 'red'], ['o', 's']):
        df_method = df_combined[df_combined['method'] == method]
        ax5.loglog(df_method['mesh_size'], df_method['rel_l2_error'] * df_method['train_time'], 
                  marker=marker, linestyle='-', color=color, label=method)
    
    ax5.set_xlabel('Mesh Size')
    ax5.set_ylabel('L2 Error × Running Time')
    ax5.set_title('Efficiency Metric')
    ax5.grid(True, which='both', ls='--', alpha=0.6)
    ax5.legend()
    
    # 6. Memory usage (if available)
    if 'memory_estimate' in df_combined.columns:
        ax6 = fig.add_subplot(gs[1, 2])
        for method, color, marker in zip(['CR-BE', 'PINN'], ['blue', 'red'], ['o', 's']):
            df_method = df_combined[df_combined['method'] == method]
            ax6.loglog(df_method['mesh_size'], df_method['memory_estimate'], 
                    marker=marker, linestyle='-', color=color, label=method)
        
        ax6.set_xlabel('Mesh Size')
        ax6.set_ylabel('Memory (KB)')
        ax6.set_title('Memory Usage')
        ax6.grid(True, which='both', ls='--', alpha=0.6)
        ax6.legend()
    
    # 7. L2 Error normalized by DOFs
    ax7 = fig.add_subplot(gs[2, 0])
    for method, color, marker in zip(['CR-BE', 'PINN'], ['blue', 'red'], ['o', 's']):
        df_method = df_combined[df_combined['method'] == method]
        dofs = df_method['number_of_collocation_points']
        ax7.loglog(dofs, df_method['rel_l2_error'], 
                  marker=marker, linestyle='-', color=color, label=method)
    
    ax7.set_xlabel('Degrees of Freedom / Collocation Points')
    ax7.set_ylabel('L2 Error')
    ax7.set_title('Error vs Problem Size')
    ax7.grid(True, which='both', ls='--', alpha=0.6)
    ax7.legend()
    
    # 8. Bar chart of convergence rates
    ax8 = fig.add_subplot(gs[2, 1])
    methods = ['CR-BE', 'PINN']
    l2_rates = [cr_l2_rate, pinn_l2_rate]
    linf_rates = [cr_linf_rate, pinn_linf_rate]
    
    x = np.arange(len(methods))
    width = 0.35
    
    ax8.bar(x - width/2, l2_rates, width, label='L2 Rate')
    ax8.bar(x + width/2, linf_rates, width, label='L∞ Rate')
    
    ax8.set_ylabel('Convergence Rate')
    ax8.set_title('Empirical Convergence Rates')
    ax8.set_xticks(x)
    ax8.set_xticklabels(methods)
    ax8.axhline(y=1.0, color='k', linestyle='--', alpha=0.5, label='First Order')
    ax8.axhline(y=2.0, color='k', linestyle='-.', alpha=0.5, label='Second Order')
    ax8.legend()
    
    # 9. Summary statistics table
    ax9 = fig.add_subplot(gs[2, 2])
    ax9.axis('off')
    
    # Create summary table
    table_data = [
        ['Method', 'CR-BE', 'PINN'],
        ['L2 Rate', f"{cr_l2_rate:.3f}", f"{pinn_l2_rate:.3f}"],
        ['L∞ Rate', f"{cr_linf_rate:.3f}", f"{pinn_linf_rate:.3f}"],
        ['Min L2 Error', f"{df_crbe['rel_l2_error'].min():.2e}", f"{df_pinn['rel_l2_error'].min():.2e}"],
        ['Min L∞ Error', f"{df_crbe['max_error'].min():.2e}", f"{df_pinn['max_error'].min():.2e}"],
        ['Max Time (s)', f"{df_crbe['train_time'].max():.2f}", f"{df_pinn['train_time'].max():.2f}"]
    ]
    
    table = ax9.table(cellText=table_data, loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)
    ax9.set_title('Summary Statistics')
    
    plt.tight_layout()
    plt.savefig('results_dashboard.png', dpi=300, bbox_inches='tight')
    plt.savefig('results_dashboard.pdf', dpi=300, bbox_inches='tight')
    plt.show()

# Create the comprehensive dashboard
create_results_dashboard()

# 8. Export results to CSV for further analysis
df_crbe.to_csv('crbe_results.csv', index=False)
df_pinn.to_csv('pinn_results.csv', index=False)
df_combined = pd.concat([
    df_crbe.assign(method='CR-BE'),
    df_pinn.assign(method='PINN')
])
df_combined.to_csv('combined_results.csv', index=False)

print("Advanced convergence analysis completed!")

## Convergence Tables

In [ ]:
memory_data = pd.DataFrame(
    {
        "cr_memory_mb": list(df_crbe["cpu_memory_usage_MB"].values),
        "pinn_memory_mb": list(df_pinn["gpu_memory_usage_MB"].values)
    }
)

In [ ]:
def generate_latex_tables(df_crbe, df_pinn, memory_data=None, sensitivity_data=None):
    """
    Generate LaTeX tables from DataFrame results.

    Args:
        df_crbe: DataFrame with CR-BE results
        df_pinn: DataFrame with PINN results
        memory_data: Optional DataFrame with memory usage
        sensitivity_data: Optional DataFrame with sensitivity analysis

    Returns:
        dict: Dictionary of LaTeX table strings
    """
    import numpy as np
    from scipy.stats import linregress

    def format_sci(x):
        if abs(x) < 0.01:
            s = f"{x:.2e}"
            base, exp = s.split('e')
            exp = int(exp)
            return f"${base}\\times 10^{{{exp}}}$"
        else:
            return f"${x:.4f}$"

    tables = {}

    mesh_sizes = df_crbe['mesh_size'].values

    log_h_crbe = np.log(1 / df_crbe['mesh_size'].values)
    log_err_l2_crbe = np.log(df_crbe['rel_l2_error'].values)
    log_err_linf_crbe = np.log(df_crbe['max_error'].values)

    log_h_pinn = np.log(1 / df_pinn['mesh_size'].values)
    log_err_l2_pinn = np.log(df_pinn['rel_l2_error'].values)
    log_err_linf_pinn = np.log(df_pinn['max_error'].values)

    crbe_l2_rate, _, crbe_l2_r2, _, _ = linregress(log_h_crbe, log_err_l2_crbe)
    crbe_linf_rate, _, crbe_linf_r2, _, _ = linregress(log_h_crbe, log_err_linf_crbe)
    pinn_l2_rate, _, pinn_l2_r2, _, _ = linregress(log_h_pinn, log_err_l2_pinn)
    pinn_linf_rate, _, pinn_linf_r2, _, _ = linregress(log_h_pinn, log_err_linf_pinn)

    # Table 1: Convergence comparison
    table1 = "\\begin{table}[htbp]\n\\centering\n"
    table1 += "\\caption{Convergence comparison of CR-BE and PINN}\n"
    table1 += "\\label{tab:convergence_comparison}\n"
    table1 += "\\begin{tabular}{ccccccc}\n\\toprule\n"
    table1 += "Mesh Size & CR-BE $L^2$ & PINN $L^2$ & CR-BE $L^\\infty$ & PINN $L^\\infty$ & CR-BE Time & PINN Time \\\\\n\\midrule\n"
    for i, mesh in enumerate(mesh_sizes):
        table1 += f"{mesh} & {format_sci(df_crbe['rel_l2_error'].iloc[i])} & {format_sci(df_pinn['rel_l2_error'].iloc[i])} & "
        table1 += f"{format_sci(df_crbe['max_error'].iloc[i])} & {format_sci(df_pinn['max_error'].iloc[i])} & "
        table1 += f"${df_crbe['train_time'].iloc[i]:.2f}$ & ${df_pinn['train_time'].iloc[i]:.2f}$ \\\\\n"
    table1 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
    tables['convergence'] = table1

    # Table 2: Convergence rates
    table2 = "\\begin{table}[htbp]\n\\centering\n"
    table2 += "\\caption{Empirical convergence rates}\n"
    table2 += "\\label{tab:convergence_rates}\n"
    table2 += "\\begin{tabular}{ccccc}\n\\toprule\n"
    table2 += "Method & $L^2$ Rate & $L^\\infty$ Rate & $R^2(L^2)$ & $R^2(L^\\infty)$ \\\\\n\\midrule\n"
    table2 += f"CR-BE & ${crbe_l2_rate:.3f}$ & ${crbe_linf_rate:.3f}$ & ${crbe_l2_r2:.3f}$ & ${crbe_linf_r2:.3f}$ \\\\\n"
    table2 += f"PINN & ${pinn_l2_rate:.3f}$ & ${pinn_linf_rate:.3f}$ & ${pinn_l2_r2:.3f}$ & ${pinn_linf_r2:.3f}$ \\\\\n"
    table2 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
    tables['rates'] = table2

    # Table 3: Computational resources
    table3 = "\\begin{table}[htbp]\n\\centering\n"
    table3 += "\\caption{Computational resource requirements}\n"
    table3 += "\\label{tab:computational_resources}\n"
    table3 += "\\begin{tabular}{ccccc}\n\\toprule\n"
    table3 += "Mesh Size & CR-BE Memory & PINN Memory & CR-BE DOFs & PINN Params \\\\\n\\midrule\n"
    for i, mesh in enumerate(mesh_sizes):
        mem_crbe = f"${memory_data['cr_memory_mb'].iloc[i]:.1f}$" if memory_data is not None else "$-$"
        mem_pinn = f"${memory_data['pinn_memory_mb'].iloc[i]:.1f}$" if memory_data is not None else "$-$"
        dofs_crbe = f"${df_crbe['number_of_collocation_points'].iloc[i]}$"
        params_pinn = f"${df_pinn['n_parameters'].iloc[i]}$" if 'n_parameters' in df_pinn.columns else "$-$"
        table3 += f"{mesh} & {mem_crbe} & {mem_pinn} & {dofs_crbe} & {params_pinn} \\\\\n"
    table3 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
    tables['resources'] = table3

    # Table 4: Efficiency (Error × Time)
    table4 = "\\begin{table}[htbp]\n\\centering\n"
    table4 += "\\caption{Efficiency: Error × Time}\n"
    table4 += "\\label{tab:efficiency}\n"
    table4 += "\\begin{tabular}{ccc}\n\\toprule\n"
    table4 += "Mesh Size & CR-BE Efficiency & PINN Efficiency \\\\\n\\midrule\n"
    for i, mesh in enumerate(mesh_sizes):
        eff_crbe = df_crbe['rel_l2_error'].iloc[i] * df_crbe['train_time'].iloc[i]
        eff_pinn = df_pinn['rel_l2_error'].iloc[i] * df_pinn['train_time'].iloc[i]
        table4 += f"{mesh} & ${eff_crbe:.2e}$ & ${eff_pinn:.2e}$ \\\\\n"
    table4 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
    tables['efficiency'] = table4

    # Table 5: Summary
    table5 = "\\begin{table}[htbp]\n\\centering\n"
    table5 += "\\caption{Summary of method performance}\n"
    table5 += "\\label{tab:summary}\n"
    table5 += "\\begin{tabular}{lcc}\n\\toprule\n"
    table5 += "Metric & CR-BE & PINN \\\\\n\\midrule\n"
    table5 += f"Minimum $L^2$ Error & {format_sci(df_crbe['rel_l2_error'].min())} & {format_sci(df_pinn['rel_l2_error'].min())} \\\\\n"
    table5 += f"Minimum $L^\\infty$ Error & {format_sci(df_crbe['max_error'].min())} & {format_sci(df_pinn['max_error'].min())} \\\\\n"
    table5 += f"Maximum Training Time (s) & ${df_crbe['train_time'].max():.2f}$ & ${df_pinn['train_time'].max():.2f}$ \\\\\n"
    table5 += f"$L^2$ Convergence Rate & ${crbe_l2_rate:.3f}$ & ${pinn_l2_rate:.3f}$ \\\\\n"
    table5 += f"$L^\\infty$ Convergence Rate & ${crbe_linf_rate:.3f}$ & ${pinn_linf_rate:.3f}$ \\\\\n"
    table5 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
    tables['summary'] = table5

    # Table 6: Memory Usage (separately)
    if memory_data is not None:
        table6 = "\\begin{table}[htbp]\n\\centering\n"
        table6 += "\\caption{Memory usage during training}\n"
        table6 += "\\label{tab:memory_usage}\n"
        table6 += "\\begin{tabular}{ccc}\n\\toprule\n"
        table6 += "Mesh Size & CR-BE (MB) & PINN (MB) \\\\\n\\midrule\n"
        for i, mesh in enumerate(mesh_sizes):
            mem_crbe = f"${memory_data['cr_memory_mb'].iloc[i]:.1f}$"
            mem_pinn = f"${memory_data['pinn_memory_mb'].iloc[i]:.1f}$"
            table6 += f"{mesh} & {mem_crbe} & {mem_pinn} \\\\\n"
        table6 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
        tables['memory'] = table6

    # Table 7: Sensitivity to diffusion coefficient
    if sensitivity_data is not None:
        table7 = "\\begin{table}[htbp]\n\\centering\n"
        table7 += "\\caption{Sensitivity to diffusion coefficient variations}\n"
        table7 += "\\label{tab:sensitivity_diffusion}\n"
        table7 += "\\begin{tabular}{ccc}\n\\toprule\n"
        table7 += "Diffusion Coefficient & CR-BE $L^2$ Error & PINN $L^2$ Error \\\\\n\\midrule\n"
        for idx, row in sensitivity_data.iterrows():
            table7 += f"${row['diffusion_coef']:.2e}$ & {format_sci(row['cr_l2_error'])} & {format_sci(row['pinn_l2_error'])} \\\\\n"
        table7 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
        tables['sensitivity'] = table7

    return tables

# Example usage
# Generate tables from your results
tables = generate_latex_tables(df_crbe, df_pinn, memory_data, df_sensitivity_data)
    
# Write tables to file
with open('convergence_tables.tex', 'w') as f:
    for name, table in tables.items():
        f.write(f"% {name}\n")
        f.write(table)
        f.write("\n\n")
            
print("LaTeX tables generated and saved to convergence_tables.tex")

In [ ]:
def generate_latex_tables(df_crbe, df_pinn, memory_data=None, sensitivity_data=None):
    """
    Generate LaTeX tables from DataFrame results
    
    Args:
        df_crbe: DataFrame with CR-BE results
        df_pinn: DataFrame with PINN results
        memory_data: Optional DataFrame with memory measurements
        sensitivity_data: Optional DataFrame with sensitivity analysis
        
    Returns:
        dict: Dictionary of LaTeX table strings
    """
    import numpy as np
    from scipy.stats import linregress
    
    # Helper function to format scientific notation nicely for LaTeX
    def format_sci(x):
        if abs(x) < 0.01:
            return f"${x:.2e}".replace("e-0", "\\times 10^{-").replace("e-", "\\times 10^{-") + "}$"
        else:
            return f"${x:.4f}$"
    
    # Calculate convergence rates
    mesh_sizes = df_crbe['mesh_size'].values
    
    log_h_crbe = np.log(1/df_crbe['mesh_size'].values)
    log_err_l2_crbe = np.log(df_crbe['rel_l2_error'].values)
    log_err_linf_crbe = np.log(df_crbe['max_error'].values)
    
    log_h_pinn = np.log(1/df_pinn['mesh_size'].values)
    log_err_l2_pinn = np.log(df_pinn['rel_l2_error'].values)
    log_err_linf_pinn = np.log(df_pinn['max_error'].values)
    
    # Linear regression to find slopes (convergence rates)
    crbe_l2_rate, _, crbe_l2_r2, _, _ = linregress(log_h_crbe, log_err_l2_crbe)
    crbe_linf_rate, _, crbe_linf_r2, _, _ = linregress(log_h_crbe, log_err_linf_crbe)
    pinn_l2_rate, _, pinn_l2_r2, _, _ = linregress(log_h_pinn, log_err_l2_pinn)
    pinn_linf_rate, _, pinn_linf_r2, _, _ = linregress(log_h_pinn, log_err_linf_pinn)
    
    # Table 1: Convergence comparison
    table1 = "\\begin{table}[htbp]\n\\centering\n"
    table1 += "\\caption{Convergence comparison of CR-BE and PINN methods}\n"
    table1 += "\\label{tab:convergence_comparison}\n"
    table1 += "\\begin{tabular}{ccccccc}\n\\toprule\n"
    table1 += "\\multirow{2}{*}{Mesh Size} & \\multicolumn{2}{c}{Relative $L^2$ Error} & "
    table1 += "\\multicolumn{2}{c}{Maximum Error ($L^\\infty$)} & \\multicolumn{2}{c}{Training Time (s)} \\\\\n"
    table1 += "\\cmidrule(lr){2-3} \\cmidrule(lr){4-5} \\cmidrule(lr){6-7}\n"
    table1 += "& CR-BE & PINN & CR-BE & PINN & CR-BE & PINN \\\\\n\\midrule\n"
    
    for i, mesh in enumerate(mesh_sizes):
        cr_l2 = format_sci(df_crbe['rel_l2_error'].iloc[i])
        pinn_l2 = format_sci(df_pinn['rel_l2_error'].iloc[i])
        cr_linf = format_sci(df_crbe['max_error'].iloc[i])
        pinn_linf = format_sci(df_pinn['max_error'].iloc[i])
        cr_time = f"${df_crbe['train_time'].iloc[i]:.2f}$"
        pinn_time = f"${df_pinn['train_time'].iloc[i]:.2f}$"
        
        table1 += f"{mesh} & {cr_l2} & {pinn_l2} & {cr_linf} & {pinn_linf} & {cr_time} & {pinn_time} \\\\\n"
    
    table1 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
    
    # Table 2: Convergence rates
    table2 = "\\begin{table}[htbp]\n\\centering\n"
    table2 += "\\caption{Empirical convergence rates for CR-BE and PINN methods}\n"
    table2 += "\\label{tab:convergence_rates}\n"
    table2 += "\\begin{tabular}{ccccc}\n\\toprule\n"
    table2 += "\\multirow{2}{*}{Method} & \\multicolumn{2}{c}{Convergence Rate} & "
    table2 += "\\multicolumn{2}{c}{Goodness of Fit ($R^2$)} \\\\\n"
    table2 += "\\cmidrule(lr){2-3} \\cmidrule(lr){4-5}\n"
    table2 += "& $L^2$ Error & $L^\\infty$ Error & $L^2$ Error & $L^\\infty$ Error \\\\\n\\midrule\n"
    
    table2 += f"CR-BE & ${crbe_l2_rate:.3f}$ & ${crbe_linf_rate:.3f}$ & "
    table2 += f"${crbe_l2_r2:.3f}$ & ${crbe_linf_r2:.3f}$ \\\\\n"
    
    table2 += f"PINN & ${pinn_l2_rate:.3f}$ & ${pinn_linf_rate:.3f}$ & "
    table2 += f"${pinn_l2_r2:.3f}$ & ${pinn_linf_r2:.3f}$ \\\\\n"
    
    table2 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
    
    # Table 3: Computational resources (if memory data available)
    table3 = "\\begin{table}[htbp]\n\\centering\n"
    table3 += "\\caption{Computational resource requirements}\n"
    table3 += "\\label{tab:computational_resources}\n"
    table3 += "\\begin{tabular}{ccccc}\n\\toprule\n"
    table3 += "\\multirow{2}{*}{Mesh Size} & \\multicolumn{2}{c}{Memory Usage (MB)} & "
    table3 += "\\multicolumn{2}{c}{DOFs / Parameters} \\\\\n"
    table3 += "\\cmidrule(lr){2-3} \\cmidrule(lr){4-5}\n"
    table3 += "& CR-BE & PINN & CR-BE & PINN \\\\\n\\midrule\n"
    
    # If we have memory data
    if memory_data is not None:
        for i, mesh in enumerate(mesh_sizes):
            mem_crbe = f"${memory_data['cr_memory_mb'].iloc[i]:.2f}$"
            mem_pinn = f"${memory_data['pinn_memory_mb'].iloc[i]:.2f}$"
            dofs_crbe = f"${df_crbe['number_of_collocation_points'].iloc[i]}$"
            
            # Check if we have parameter counts for PINN
            if 'n_parameters' in df_pinn.columns:
                params_pinn = f"${df_pinn['n_parameters'].iloc[i]}$"
            else:
                params_pinn = "$-$"
                
            table3 += f"{mesh} & {mem_crbe} & {mem_pinn} & {dofs_crbe} & {params_pinn} \\\\\n"
    else:
        # If no memory data, use placeholder or skip
        for i, mesh in enumerate(mesh_sizes):
            dofs_crbe = f"${df_crbe['number_of_collocation_points'].iloc[i]}$"
            
            # Check if we have parameter counts for PINN
            if 'n_parameters' in df_pinn.columns:
                params_pinn = f"${df_pinn['n_parameters'].iloc[i]}$"
            else:
                params_pinn = "$-$"
                
            table3 += f"{mesh} & $-$ & $-$ & {dofs_crbe} & {params_pinn} \\\\\n"
    
    table3 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
    
    # Table 4: Efficiency comparison
    table4 = "\\begin{table}[htbp]\n\\centering\n"
    table4 += "\\caption{Efficiency comparison ($L^2$ error $\\times$ training time)}\n"
    table4 += "\\label{tab:efficiency_comparison}\n"
    table4 += "\\begin{tabular}{ccc}\n\\toprule\n"
    table4 += "Mesh Size & CR-BE Efficiency & PINN Efficiency \\\\\n\\midrule\n"
    
    for i, mesh in enumerate(mesh_sizes):
        eff_crbe = df_crbe['rel_l2_error'].iloc[i] * df_crbe['train_time'].iloc[i]
        eff_pinn = df_pinn['rel_l2_error'].iloc[i] * df_pinn['train_time'].iloc[i]
        
        table4 += f"{mesh} & ${eff_crbe:.2e}$ & ${eff_pinn:.2e}$ \\\\\n"
    
    table4 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
    
    # Table 5: Summary statistics
    table5 = "\\begin{table}[htbp]\n\\centering\n"
    table5 += "\\caption{Summary of method performance}\n"
    table5 += "\\label{tab:summary_statistics}\n"
    table5 += "\\begin{tabular}{lcc}\n\\toprule\n"
    table5 += "Metric & CR-BE & PINN \\\\\n\\midrule\n"
    
    min_l2_crbe = format_sci(df_crbe['rel_l2_error'].min())
    min_l2_pinn = format_sci(df_pinn['rel_l2_error'].min())
    min_linf_crbe = format_sci(df_crbe['max_error'].min())
    min_linf_pinn = format_sci(df_pinn['max_error'].min())
    max_time_crbe = f"${df_crbe['train_time'].max():.2f}$"
    max_time_pinn = f"${df_pinn['train_time'].max():.2f}$"
    
    table5 += f"Minimum $L^2$ Error & {min_l2_crbe} & {min_l2_pinn} \\\\\n"
    table5 += f"Minimum $L^\\infty$ Error & {min_linf_crbe} & {min_linf_pinn} \\\\\n"
    table5 += f"Maximum Training Time (s) & {max_time_crbe} & {max_time_pinn} \\\\\n"
    table5 += f"$L^2$ Convergence Rate & ${crbe_l2_rate:.3f}$ & ${pinn_l2_rate:.3f}$ \\\\\n"
    table5 += f"$L^\\infty$ Convergence Rate & ${crbe_linf_rate:.3f}$ & ${pinn_linf_rate:.3f}$ \\\\\n"
    
    # Determine scaling based on convergence rate
    scaling_crbe = f"$O(n^{{{abs(crbe_l2_rate):.1f}}})$"
    scaling_pinn = f"$O(n^{{{abs(pinn_l2_rate):.1f}}})$"
    table5 += f"Error Scaling & {scaling_crbe} & {scaling_pinn} \\\\\n"
    
    table5 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
    
    # Table 6: Method characteristics
    table6 = "\\begin{table}[htbp]\n\\centering\n"
    table6 += "\\caption{Quantitative evidence for method characteristics}\n"
    table6 += "\\label{tab:method_characteristics}\n"
    table6 += "\\begin{tabular}{lcc}\n\\toprule\n"
    table6 += "Characteristic & CR-BE & PINN \\\\\n\\midrule\n"
    
    # Use data for mesh size 64 (index 4) as reference point
    mesh_64_idx = list(mesh_sizes).index(64) if 64 in mesh_sizes else -2  # Second to last as fallback
    
    eff_crbe_64 = df_crbe['rel_l2_error'].iloc[mesh_64_idx] * df_crbe['train_time'].iloc[mesh_64_idx]
    eff_pinn_64 = df_pinn['rel_l2_error'].iloc[mesh_64_idx] * df_pinn['train_time'].iloc[mesh_64_idx]
    
    table6 += f"Accuracy (Best $L^2$ Error) & {min_l2_crbe} & {min_l2_pinn} \\\\\n"
    table6 += f"Computational Efficiency (Time for mesh=64) & ${df_crbe['train_time'].iloc[mesh_64_idx]:.2f}$ s & ${df_pinn['train_time'].iloc[mesh_64_idx]:.2f}$ s \\\\\n"
    
    if memory_data is not None:
        table6 += f"Memory Usage (MB for mesh=64) & ${memory_data['cr_memory_mb'].iloc[mesh_64_idx]:.2f}$ & ${memory_data['pinn_memory_mb'].iloc[mesh_64_idx]:.2f}$ \\\\\n"
    else:
        table6 += f"Memory Usage (MB for mesh=64) & $-$ & $-$ \\\\\n"
        
    table6 += f"Convergence Rate ($L^2$) & ${crbe_l2_rate:.3f}$ & ${pinn_l2_rate:.3f}$ \\\\\n"
    table6 += f"Error/Cost Ratio (mesh=64) & ${eff_crbe_64:.2e}$ & ${eff_pinn_64:.2e}$ \\\\\n"
    
    table6 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
    
    # Table 7: Parameter sensitivity (if available)
    table7 = ""
    if sensitivity_data is not None:
        table7 = "\\begin{table}[htbp]\n\\centering\n"
        table7 += "\\caption{Sensitivity to diffusion coefficient}\n"
        table7 += "\\label{tab:parameter_sensitivity}\n"
        table7 += "\\begin{tabular}{cccccc}\n\\toprule\n"
        table7 += "\\multirow{2}{*}{Diffusion ($D$)} & \\multirow{2}{*}{Mesh Size} & "
        table7 += "\\multicolumn{2}{c}{$L^2$ Error} & \\multicolumn{2}{c}{Training Time (s)} \\\\\n"
        table7 += "\\cmidrule(lr){3-4} \\cmidrule(lr){5-6}\n"
        table7 += "& & CR-BE & PINN & CR-BE & PINN \\\\\n\\midrule\n"
        
        # For each diffusion value and selected mesh sizes
        diffusion_values = sensitivity_data['diffusion_coef'].unique()
        mesh_selection = [32]  # Example: focus on mesh size 32
        
        for d in diffusion_values:
            for mesh in mesh_selection:
                df_d_mesh_cr = sensitivity_data[(sensitivity_data['diffusion_coef'] == d) & 
                                             (sensitivity_data['mesh_size'] == mesh) &
                                             (sensitivity_data['method'] == 'CR-BE')]
                
                df_d_mesh_pinn = sensitivity_data[(sensitivity_data['diffusion_coef'] == d) & 
                                               (sensitivity_data['mesh_size'] == mesh) &
                                               (sensitivity_data['method'] == 'PINN')]
                
                if len(df_d_mesh_cr) > 0 and len(df_d_mesh_pinn) > 0:
                    cr_l2 = format_sci(df_d_mesh_cr['rel_l2_error'].iloc[0])
                    pinn_l2 = format_sci(df_d_mesh_pinn['rel_l2_error'].iloc[0])
                    cr_time = f"${df_d_mesh_cr['train_time'].iloc[0]:.2f}$"
                    pinn_time = f"${df_d_mesh_pinn['train_time'].iloc[0]:.2f}$"
                    
                    table7 += f"{d} & {mesh} & {cr_l2} & {pinn_l2} & {cr_time} & {pinn_time} \\\\\n"
        
        table7 += "\\bottomrule\n\\end{tabular}\n\\end{table}"
    
    tables = {
        "convergence_comparison": table1,
        "convergence_rates": table2,
        "computational_resources": table3,
        "efficiency_comparison": table4,
        "summary_statistics": table5,
        "method_characteristics": table6
    }
    
    if table7:
        tables["parameter_sensitivity"] = table7
        
    return tables

# Example usage
# Generate tables from your results
tables = generate_latex_tables(df_crbe, df_pinn)
    
# Write tables to file
with open('convergence_tables_2.tex', 'w') as f:
    for name, table in tables.items():
        f.write(f"% {name}\n")
        f.write(table)
        f.write("\n\n")
            
print("LaTeX tables generated and saved to convergence_tables_2.tex")

In [ ]:
%cd ..

In [ ]:
!tar -czvf AirPollution.tar.gz AirPollution

In [ ]:
%cd AirPollution